**Important: This notebook will only work with fastai-0.7.x. Do not try to run any fastai-1.x code from this path in the repository because it will load fastai-0.7.x**

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.nlp import *
from sklearn.linear_model import LogisticRegression

## IMDB dataset and the sentiment classification task

The [large movie review dataset](http://ai.stanford.edu/~amaas/data/sentiment/) contains a collection of 50,000 reviews from IMDB. The dataset contains an even number of positive and negative reviews. The authors considered only highly polarized reviews. A negative review has a score ≤ 4 out of 10, and a positive review has a score ≥ 7 out of 10. Neutral reviews are not included in the dataset. The dataset is divided into training and test sets. The training set is the same 25,000 labeled reviews.

The **sentiment classification task** consists of predicting the polarity (positive or negative) of a given text.

To get the dataset, in your terminal run the following commands:

`wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz`

`gunzip aclImdb_v1.tar.gz`

`tar -xvf aclImdb_v1.tar`

### Tokenizing and term document matrix creation

In [2]:
PATH='data/aclImdb/'
names = ['neg','pos']

In [8]:
%ls {PATH}

imdbEr.txt  imdb.vocab  README  test/  train/


In [9]:
%ls {PATH}train

labeledBow.feat  pos/    unsupBow.feat  urls_pos.txt
neg/             unsup/  urls_neg.txt   urls_unsup.txt


In [10]:
%ls {PATH}train/pos | head

0_9.txt
10000_8.txt
10001_10.txt
10002_7.txt
10003_8.txt
10004_8.txt
10005_7.txt
10006_7.txt
10007_7.txt
10008_7.txt
ls: write error: Broken pipe


In [11]:
trn,trn_y = texts_labels_from_folders(f'{PATH}train',names)
val,val_y = texts_labels_from_folders(f'{PATH}test',names)

Here is the text of the first review

In [28]:
??texts_labels_from_folders

In [13]:
trn[0]

'I disagree with much that has been written and said about this supposed "masterpiece" of the German New Wave: <br /><br />1) There are major flaws in simple exposition, in the basic communication of critical plot points, as relating to Maria\'s abortion and the secret contract between Oswald and her husband. How many viewers understood that the husband agreed, in exchange for substantial financial remuneration, not to return to and reclaim his wife until Oswald was dead?<br /><br />2) The ending is highly unsatisfying because arbitrary and accidental. The original screenplay called for Maria to commit suicide after the reading of Oswald\'s will, on finding out that her husband had in effect sold their marriage to Oswald. In the final version, however, Maria only runs water from a faucet across her wrist in a gesture of suicide. Maria is then summarily blown up, rather than having to confront and live with the consequences of her self-delusion and moral compromise.<br /><br />3) Fassbi

In [14]:
trn_y[0]

0

[`CountVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) converts a collection of text documents to a matrix of token counts (part of `sklearn.feature_extraction.text`).

In [15]:
veczr = CountVectorizer(tokenizer=tokenize)

`fit_transform(trn)` finds the vocabulary in the training set. It also transforms the training set into a term-document matrix. Since we have to apply the *same transformation* to your validation set, the second line uses just the method `transform(val)`. `trn_term_doc` and `val_term_doc` are sparse matrices. `trn_term_doc[i]` represents training document i and it contains a count of words for each document for each word in the vocabulary.

In [18]:
trn_term_doc = veczr.fit_transform(trn)
val_term_doc = veczr.transform(val)

In [19]:
trn_term_doc

<25000x75132 sparse matrix of type '<class 'numpy.int64'>'
	with 3749745 stored elements in Compressed Sparse Row format>

In [20]:
trn_term_doc[0]

<1x75132 sparse matrix of type '<class 'numpy.int64'>'
	with 237 stored elements in Compressed Sparse Row format>

In [21]:
vocab = veczr.get_feature_names(); vocab[5000:5005]

['aussie', 'aussies', 'austen', 'austeniana', 'austens']

In [22]:
w0 = set([o.lower() for o in trn[0].split(' ')]); w0

{'"experts"',
 '"masterpiece"',
 '--',
 '/>1)',
 '/>2)',
 '/>3)',
 '/>4)',
 '/><br',
 '/>strangely,',
 '/>the',
 '<br',
 'a',
 'abortion',
 'about',
 'accidental.',
 'across',
 'acting.<br',
 'addiction',
 'after',
 'ages.',
 'agreed,',
 'amateurishness',
 'ambition',
 'amnesia',
 'and',
 'arbitrary',
 'are',
 'as',
 'assert,',
 'asserting',
 'basic',
 'be',
 'because',
 'been',
 'being',
 'between',
 'blown',
 'but',
 'by',
 'called',
 'clumsily',
 'collective',
 'commit',
 'communication',
 'compromise.<br',
 'condemning',
 'confront',
 'consequences',
 'consequent',
 'contract',
 'contrary',
 'country',
 'critical',
 'critics',
 'dead?<br',
 'disagree',
 'does',
 "don't",
 'drama.',
 'dramatic',
 'drug',
 'economic',
 'effect',
 'effect.',
 'ending',
 'events',
 'ever',
 'exchange',
 'exposition,',
 'fassbinder',
 "fassbinder's",
 'faucet',
 'final',
 'financial',
 'finding',
 'flaws',
 'for',
 'forcibly',
 'from',
 'german',
 'germany.',
 'gesture',
 'had',
 'has',
 'have',
 'havin

In [23]:
len(w0)

232

In [24]:
veczr.vocabulary_['absurd']

1297

In [25]:
trn_term_doc[0,1297]

0

In [26]:
trn_term_doc[0,5000]

0

## Naive Bayes

We define the **log-count ratio** $r$ for each word $f$:

$r = \log \frac{\text{ratio of feature $f$ in positive documents}}{\text{ratio of feature $f$ in negative documents}}$

where ratio of feature $f$ in positive documents is the number of times a positive document has a feature divided by the number of positive documents.

In [29]:
def pr(y_i):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

In [30]:
x=trn_term_doc
y=trn_y

r = np.log(pr(1)/pr(0))
b = np.log((y==1).mean() / (y==0).mean())

Here is the formula for Naive Bayes.

In [31]:
pre_preds = val_term_doc @ r.T + b
preds = pre_preds.T>0
(preds==val_y).mean()

0.81656

...and binarized Naive Bayes.

In [32]:
x=trn_term_doc.sign()
r = np.log(pr(1)/pr(0))

pre_preds = val_term_doc.sign() @ r.T + b
preds = pre_preds.T>0
(preds==val_y).mean()

0.83016

### Logistic regression

Here is how we can fit logistic regression where the features are the unigrams.

In [33]:
m = LogisticRegression(C=1e8, dual=True)
m.fit(x, y)
preds = m.predict(val_term_doc)
(preds==val_y).mean()

/home/sidd/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/sidd/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.8324

In [34]:
m = LogisticRegression(C=1e8, dual=True)
m.fit(trn_term_doc.sign(), y)
preds = m.predict(val_term_doc.sign())
(preds==val_y).mean()

0.85516

...and the regularized version

In [35]:
m = LogisticRegression(C=0.1, dual=True)
m.fit(x, y)
preds = m.predict(val_term_doc)
(preds==val_y).mean()

0.84872

In [36]:
m = LogisticRegression(C=0.1, dual=True)
m.fit(trn_term_doc.sign(), y)
preds = m.predict(val_term_doc.sign())
(preds==val_y).mean()

0.88404

### Trigram with NB features

Our next model is a version of logistic regression with Naive Bayes features described [here](https://www.aclweb.org/anthology/P12-2018). For every document we compute binarized features as described above, but this time we use bigrams and trigrams too. Each feature is a log-count ratio. A logistic regression model is then trained to predict sentiment.

In [37]:
veczr =  CountVectorizer(ngram_range=(1,3), tokenizer=tokenize, max_features=800000)
trn_term_doc = veczr.fit_transform(trn)
val_term_doc = veczr.transform(val)

In [38]:
trn_term_doc.shape

(25000, 800000)

In [39]:
vocab = veczr.get_feature_names()

In [40]:
vocab[200000:200005]

['by vast', 'by vengeance', 'by vengeance .', 'by vera', 'by vera miles']

In [41]:
y=trn_y
x=trn_term_doc.sign()
val_x = val_term_doc.sign()

In [42]:
r = np.log(pr(1) / pr(0))
b = np.log((y==1).mean() / (y==0).mean())

Here we fit regularized logistic regression where the features are the trigrams.

In [43]:
m = LogisticRegression(C=0.1, dual=True)
m.fit(x, y);

preds = m.predict(val_x)
(preds.T==val_y).mean()

/home/sidd/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.905

Here is the $\text{log-count ratio}$ `r`.  

In [44]:
r.shape, r

((1, 800000),
 matrix([[-0.05468, -0.161  , -0.24784, ...,  1.09861, -0.69315, -0.69315]]))

In [45]:
np.exp(r)

matrix([[0.94678, 0.85129, 0.78049, ..., 3.     , 0.5    , 0.5    ]])

Here we fit regularized logistic regression where the features are the trigrams' log-count ratios.

In [46]:
x_nb = x.multiply(r)
m = LogisticRegression(dual=True, C=0.1)
m.fit(x_nb, y);

val_x_nb = val_x.multiply(r)
preds = m.predict(val_x_nb)
(preds.T==val_y).mean()

0.91768

## fastai NBSVM++

sl ... max sentence length

In [48]:
sl=2000

In [49]:
# Here is how we get a model from a bag of words
md = TextClassifierData.from_bow(trn_term_doc, trn_y, val_term_doc, val_y, sl)

In [50]:
learner = md.dotprod_nb_learner()
learner.fit(0.02, 1, wds=1e-6, cycle_len=1)

epoch      trn_loss   val_loss   <lambda>                     
    0      0.02507501437651315 [0.11947]  0.9167600000190735



[array([0.11947]), 0.9167600000190735]

In [51]:
learner.fit(0.02, 2, wds=1e-6, cycle_len=1)

epoch      trn_loss   val_loss   <lambda>                     
    0      0.02127116495933404 [0.11349]  0.9208400000190735
    1      0.011063079297005058 [0.11142]  0.9217200000190735 



[array([0.11142]), 0.9217200000190735]

In [52]:
learner.fit(0.02, 2, wds=1e-6, cycle_len=1)

epoch      trn_loss   val_loss   <lambda>                     
    0      0.01881257667500745 [0.11001]  0.9222800000190735
    1      0.010419971695920082 [0.10903]  0.9222800000190735  



[array([0.10903]), 0.9222800000190735]

## References

* Baselines and Bigrams: Simple, Good Sentiment and Topic Classification. Sida Wang and Christopher D. Manning [pdf](https://www.aclweb.org/anthology/P12-2018)